In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.optimize import curve_fit
import h5py
import pyvisa
import time
from scipy.signal import savgol_filter
import scipy.signal as signal
from scipy import interpolate
from matplotlib.ticker import MaxNLocator
from scipy.optimize import least_squares

In [2]:
factor = 2
SMALL_SIZE = factor*8
MEDIUM_SIZE = factor*10
BIGGER_SIZE = factor*12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
rm = pyvisa.ResourceManager()
rm.list_resources()

('USB0::0x1AB1::0x04CE::DS1ZA204519116::INSTR',
 'USB0::0x0957::0x2607::MY59001082::INSTR',
 'ASRL4::INSTR',
 'ASRL5::INSTR',
 'ASRL6::INSTR',
 'GPIB0::22::INSTR')

In [4]:
for i in range(2):
    try:
        HV_control = rm.open_resource('ASRL6::INSTR')
        print(HV_control.query("?"))
    except pyvisa.VisaIOError:
        pass

HV_control v1.3 ready.



In [5]:
DMM = rm.open_resource('GPIB0::22::INSTR')
DMM.read_termination = "\r\n"
DMM.timeout = 5000

### Data collection functions

In [6]:
def get_data(t_total, do_tqdm=True, NPLC=10):
    # start the measurement
    DMM.clear()
    DMM.write(f"NDIG 8")
    DMM.write(f"NPLC {NPLC}")
    DMM.write("TARM AUTO")
    
    # collect data
    data = []
    t0 = time.time()
    if do_tqdm:
        pbar = tqdm(total=t_total)

    while time.time()-t0 < t_total:
        data.append([time.time(), float(DMM.read())])
        print(f"{data[-1][1]:.9f}", end='\r')
        if do_tqdm:
            pbar.update(int(time.time()-t0)+1 - pbar.n)

    if do_tqdm:
        pbar.close()
    data = np.array(data)
    
    return data
    
def DMM_to_HDF(run_name, t_total, fname="test.hdf", overwrite=False,
               C_LPF_uF=22, do_tqdm=False, NPLC=10):
    # check that a dataset with this run_name doesn't already exist
    try:
        with h5py.File(fname, 'a') as f:
            if run_name in f.keys():
                if overwrite:
                    del f[run_name]
                else:
                    print("ERROR: Run name already exists.")
                    return
    except OSError:
        pass
    
    # get data
    data = get_data(t_total, do_tqdm, NPLC)
    
    # save to HDF
    with h5py.File(fname, 'a') as f:
        dset = f.create_dataset(run_name, data=data)
        dset.attrs['timestamp'] = time.time()
        dset.attrs['overwritten'] = str(overwrite)
        dset.attrs['t_total'] = t_total
        dset.attrs['C_LPF_uF'] = C_LPF_uF
        dset.attrs['DMM_temp'] = float(DMM.query("TEMP?"))
        dset.attrs['NPLC'] = NPLC

### Testing

In [7]:
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
print(DMM.query("TEMP?"))
try:
    DMM.write("ACAL DCV")
except pyvisa.errors.VisaIOError:
    pass

2023-03-15 17:14:43
39.5


In [10]:
HV_control.write("r0");
HV_control.write("d0");

In [11]:
try:
    for i in tqdm(range(99999)):
        DMM_to_HDF(
            run_name  = f"O-{i}",
            t_total   = 3600,
            overwrite = False,
            C_LPF_uF  = 1000,
            do_tqdm   = False,
            NPLC      = 10,
        )
except KeyboardInterrupt:
    pass

  0%|          | 0/99999 [00:00<?, ?it/s]

VisaIOError: VI_ERROR_INV_RSRC_NAME (-1073807342): Invalid resource reference specified. Parsing error.